# LSTM

In [414]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [415]:
data = pd.read_excel("original_data.xlsx")
data

,Age,Gender,Heart Rate,Respiratory Rate,Temperature,SpO2,Health Condition
0,2,M,96,20,98.6,99,Unprovoked Seizure
1,2,M,102,22,98.6,99,Unprovoked Seizure
2,2,M,84,24,97.4,99,Unprovoked Seizure
3,2,M,137,24,99.6,98,Viral Fever
4,2,M,122,26,98.6,99,Viral Fever
...,...,...,...,...,...,...,...
183,2,F,118,26,98.6,98,Complex Febrile Seizure/Viral Fever
184,2,F,118,28,98.2,97,Complex Febrile Seizure/Viral Fever
185,4,M,100,28,98.6,99,Genetic Epilepsywith Febrile Seizure
186,4,M,98,28,98.7,99,Genetic Epilepsywith Febrile Seizure


In [416]:
# Define the input features and target variable
features = data[['Heart Rate', 'Respiratory Rate']].values
spo2 = data['SpO2'].values

In [417]:
# Normalize the input features
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

In [418]:

# Reshape input features for LSTM
time_steps = 1  # You can adjust the number of time steps as needed
features = features.reshape(-1, time_steps, 2)

In [419]:
# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, spo2, test_size=0.2, random_state=42)

In [420]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 2)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [421]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [422]:
# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
3/3 [==============================] - 9s 917ms/step - loss: 9672.5342 - val_loss: 9706.6328
Epoch 2/100
3/3 [==============================] - 0s 37ms/step - loss: 9665.8301 - val_loss: 9699.5527
Epoch 3/100
3/3 [==============================] - 0s 37ms/step - loss: 9658.7793 - val_loss: 9691.8984
Epoch 4/100
3/3 [==============================] - 0s 33ms/step - loss: 9651.1084 - val_loss: 9683.4238
Epoch 5/100
3/3 [==============================] - 0s 36ms/step - loss: 9642.5938 - val_loss: 9673.8604
Epoch 6/100
3/3 [==============================] - 0s 33ms/step - loss: 9632.9502 - val_loss: 9662.9297
Epoch 7/100
3/3 [==============================] - 0s 42ms/step - loss: 9621.9561 - val_loss: 9650.3193
Epoch 8/100
3/3 [==============================] - 0s 34ms/step - loss: 9609.2412 - val_loss: 9635.6709
Epoch 9/100
3/3 [==============================] - 0s 36ms/step - loss: 9594.5391 - val_loss: 9618.5459
Epoch 10/100
3/3 [==============================] - 0s 34ms/ste

Epoch 80/100
3/3 [==============================] - 0s 32ms/step - loss: 77.2836 - val_loss: 101.6659
Epoch 81/100
3/3 [==============================] - 0s 31ms/step - loss: 76.7950 - val_loss: 101.0512
Epoch 82/100
3/3 [==============================] - 0s 34ms/step - loss: 76.3029 - val_loss: 100.4915
Epoch 83/100
3/3 [==============================] - 0s 34ms/step - loss: 75.7711 - val_loss: 99.9038
Epoch 84/100
3/3 [==============================] - 0s 33ms/step - loss: 75.2706 - val_loss: 99.3291
Epoch 85/100
3/3 [==============================] - 0s 35ms/step - loss: 74.8366 - val_loss: 98.7505
Epoch 86/100
3/3 [==============================] - 0s 32ms/step - loss: 74.2783 - val_loss: 98.0273
Epoch 87/100
3/3 [==============================] - 0s 33ms/step - loss: 73.8152 - val_loss: 97.3065
Epoch 88/100
3/3 [==============================] - 0s 34ms/step - loss: 73.2916 - val_loss: 96.6410
Epoch 89/100
3/3 [==============================] - 0s 33ms/step - loss: 72.8274 - val_l

In [423]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Mean Squared Error (MSE) on Test Data: {loss}")

2/2 [==============================] - 0s 11ms/step - loss: 89.6222
Mean Squared Error (MSE) on Test Data: 89.62220001220703


In [424]:
# Make predictions
y_pred = model.predict(X_test)

In [425]:
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

Mean Absolute Error (MAE): 7.169075413754112
Mean Squared Error (MSE): 89.62220138519402


In [426]:
# Define a threshold for classifying SpO2 as low, normal, or high (adjust as needed)
threshold_low = 90
threshold_high = 95

# Classify predicted SpO2 values
y_pred_class = np.where(y_pred < threshold_low, 'Low',
                         np.where(y_pred < threshold_high, 'Normal', 'High'))

y_test_class = np.where(y_test < threshold_low, 'Low',
                        np.where(y_test < threshold_high, 'Normal', 'High'))


In [427]:
# Calculate Accuracy
accuracy = accuracy_score(y_test_class, y_pred_class)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7894736842105263


# Hyper Parameter Tuning

In [428]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score

# Load your preprocessed dataset
# Replace 'X' and 'y' with your features and target variable
X = data[['Heart Rate', 'Respiratory Rate']].values  # Features
y = data['SpO2'].values  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data
n_samples, n_features = X_train.shape
X_train = X_train.reshape((n_samples, 1, n_features))
X_test = X_test.reshape((X_test.shape[0], 1, n_features))

# Define the LSTM model
def create_lstm_model(units=50):
    model = Sequential()
    model.add(LSTM(units, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Create a KerasRegressor wrapper for Scikit-learn
model = KerasRegressor(build_fn=create_lstm_model)

# Define hyperparameters to tune
param_grid = {
    'units': [50, 100, 150],  # Units in LSTM layer
    'batch_size': [16, 32, 64],  # Batch size for training
}

# Create grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
best_model = create_lstm_model(units=best_params['units'])
best_model.fit(X_train, y_train, epochs=100, batch_size=best_params['batch_size'], verbose=1)

# Make predictions
y_pred = best_model.predict(X_test)

# Calculate and print evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Best hyperparameters: {best_params}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Accuracy: {accuracy:.2%}")


<ipython-input-428-f01e7aaee821>:31: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasRegressor(build_fn=create_lstm_model)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
10/10 [==============================] - 3s 4ms/step - loss: 9199.9131
Epoch 1/100
10/10 [==============================] - 3s 6ms/step - loss: 9408.2764
Epoch 2/100
10/10 [==============================] - 0s 7ms/step - loss: 8387.7197
Epoch 3/100
10/10 [==============================] - 0s 5ms/step - loss: 7357.4731
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 6294.0210
Epoch 5/100
10/10 [==============================] - 0s 4ms/step - loss: 4921.1816
Epoch 6/100
10/10 [==============================] - 0s 4ms/step - loss: 3699.1138
Epoch 7/100
10/10 [==============================] - 0s 3ms/step - loss: 2820.8870
Epoch 8/100
10/10 [==============================] - 0s 4ms/step - loss: 2032.2332
Epoch 9/100
10/10 [==============================] - 0s 4ms/step - loss: 1383.1339
Epoch 10/100
10/10 [==============================] - 0s 4ms/step - loss: 831.9114
Epoch 11/100
10/10 [===================

10/10 [==============================] - 0s 4ms/step - loss: 134.9176
Epoch 53/100
10/10 [==============================] - 0s 4ms/step - loss: 133.9486
Epoch 54/100
10/10 [==============================] - 0s 3ms/step - loss: 132.4002
Epoch 55/100
10/10 [==============================] - 0s 4ms/step - loss: 132.0211
Epoch 56/100
10/10 [==============================] - 0s 5ms/step - loss: 121.1640
Epoch 57/100
10/10 [==============================] - 0s 4ms/step - loss: 121.7530
Epoch 58/100
10/10 [==============================] - 0s 4ms/step - loss: 116.4210
Epoch 59/100
10/10 [==============================] - 0s 4ms/step - loss: 112.8648
Epoch 60/100
10/10 [==============================] - 0s 4ms/step - loss: 109.0162
Epoch 61/100
10/10 [==============================] - 0s 4ms/step - loss: 106.3728
Epoch 62/100
10/10 [==============================] - 0s 5ms/step - loss: 107.1733
Epoch 63/100
10/10 [==============================] - 0s 4ms/step - loss: 100.9152
Epoch 64/100
10/1

In [429]:
# Define a threshold for classifying SpO2 as low, normal, or high (adjust as needed)
threshold_low = 90
threshold_high = 95

In [430]:
# Classify predicted SpO2 values
y_pred_class = np.where(y_pred < threshold_low, 'Low',
                         np.where(y_pred < threshold_high, 'Normal', 'High'))

y_test_class = np.where(y_test < threshold_low, 'Low',
                        np.where(y_test < threshold_high, 'Normal', 'High'))


In [431]:
# Calculate Accuracy
accuracy = accuracy_score(y_test_class, y_pred_class)

In [439]:
print(f"Best hyperparameters: {best_params}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Accuracy: {accuracy:.2%}")

Best hyperparameters: {'batch_size': 16, 'units': 50}
Mean Squared Error (MSE): 32.44
Mean Absolute Error (MAE): 4.24
Accuracy: 78.95%


In [440]:
# Save the trained model
best_model.save('lstm_model.h5')